In [15]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [12]:
pc = Pinecone(api_key="a72bc5cc-7895-405e-b49b-3cbbc9c9528b")
pc.create_index(
    name="rag", dimension = 1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)


In [13]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Zhao',
  'subject': 'Computer Science',
  'stars': 4,
  'review': "Dr. Zhao is an excellent professor who really knows her stuff. Her lectures are engaging and she's always willing to help students during office hours."},
 {'professor': 'Professor Michael Hernandez',
  'subject': 'Chemistry',
  'stars': 3,
  'review': 'Professor Hernandez is knowledgeable, but his lectures can be a bit dry at times. The labs are interesting, but the grading on assignments and exams is quite harsh.'},
 {'professor': 'Dr. Olivia Nguyen',
  'subject': 'Psychology',
  'stars': 5,
  'review': "Dr. Nguyen is by far one of the best professors I've had. Her passion for the subject is contagious, and she does a great job of making the material interesting and relevant. I highly recommend taking any of her courses."},
 {'professor': 'Professor David Lee',
  'subject': 'Economics',
  'stars': 2,
  'review': 'I would not recommend taking a course with Professor Lee. His lectures are disor

In [16]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"],
        } 
    })

In [17]:
processed_data[0]

{'values': [-0.0018267535,
  9.638348e-05,
  0.004406102,
  0.044663098,
  0.0122604575,
  -0.012829693,
  -0.018708145,
  0.052895118,
  -0.042889707,
  -0.016803395,
  0.0019458004,
  0.03008191,
  -0.0038943375,
  -0.00469893,
  0.012052468,
  0.041576087,
  -0.03713167,
  0.0015079268,
  0.023010252,
  0.020175021,
  0.00471535,
  -0.0021715788,
  0.04267077,
  -0.040153,
  -0.039058316,
  -0.052544817,
  0.036847055,
  0.0015530825,
  -0.0049835476,
  0.005451525,
  0.07325623,
  -0.0056321477,
  0.004671563,
  -0.009485435,
  -0.038029313,
  0.03492041,
  -0.013574079,
  -0.0021277915,
  0.017679142,
  -0.031417422,
  0.0022523117,
  0.006486001,
  -0.02484932,
  0.033978984,
  0.032774832,
  -0.0023973573,
  -0.03485473,
  0.015686817,
  0.036803268,
  0.06244076,
  -0.056573257,
  -0.00016488673,
  0.002480827,
  0.0046004085,
  -0.035949413,
  0.01737263,
  0.024717959,
  0.04610808,
  0.0044964133,
  -0.025068257,
  0.025878323,
  -0.010218873,
  -0.010853789,
  -0.0035522487

In [18]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1",
    
)

{'upserted_count': 21}

In [19]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 21}},
 'total_vector_count': 21}